In [25]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [26]:
!python -V

Python 3.12.4


In [27]:
import pickle
import pandas as pd

In [28]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [29]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [30]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [31]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [32]:
import numpy as np

print(np.std(y_pred))

6.247488852238703


In [35]:
year = 2023
month = 3

output_file = "results.parquet"

df_result = df.copy()

df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result['y_pred'] = pd.Series(y_pred)
df_result = df_result[['ride_id', 'y_pred']]

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)